# Credit Card Retention Analysis

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot
sns.set()
pd.options.display.max_columns = 999

In [80]:
data = pd.read_csv('../data/BankChurners_v2.csv')

In [10]:
data = data[['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',]]

In [14]:
data['Education_Level'] = data['Education_Level'].fillna('Unknown')
data['Marital_Status'] = data['Marital_Status'].fillna('Unknown')
data['Income_Category'] = data['Income_Category'].fillna('Unknown')

In [18]:
# https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-5-binning-c5bd5fd1b950
bins = [25, 30, 40, 50, 60, 70, 80]
labels = ['20s', '30s', '40s', '50s', '60s', '70s']
data['Customer_Age_bins'] = pd.cut(data['Customer_Age'], bins=bins, labels=labels, include_lowest=True, right=False)

***

## Summary Statistics

To start to understand our dataset a bit more, we will want to produce **summary** or **descriptive statistics** that can tell us more about the variables and features within the dataset. Typically, we are looking to understand:

    1) how many instances are in the dataset (frequency or counts) 
    2) a measure of central tendency (mean, median, mode)
    3) the spread of the dataset (variance, standard deviation)

Depending on what variable or feature you are looking at, a different measure of central tendency may help represent that data better. It's important to understand the differences between them and when to use which so you can make the right choice. 

The **Mean** is the average of all values in a dataset, while the **Median** represents the midpoint of the values (50% above and 50% below. For example, you have a list y consisting of numbers 0-100 in intervals of 10 as below: 

In [23]:
y = list(range(0, 110, 10))
print(y)

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


Then the mean and median are both equal as shown below: 

In [24]:
mean = sum(y)/len(y) # the way to get the count of a list is by using the len() method
print(mean)

50.0


In [25]:
np.mean(y)

50.0

In [26]:
np.median(y)

50.0

If I add one more data point to the set, 900, let's see how things change:

In [27]:
y.append(900)

In [28]:
np.mean(y)

120.83333333333333

In [29]:
np.median(y)

55.0

Notice how the **Mean** jumped up, highly impacted by that new data point (outlier), while the median came up slightly, but was much less sensitive. In the case where there are outliers, the **Median** is a much better representation of central tendency than the **Mean**.

In python, we can use the `.describe()` method to see these metrics for all the numerical variables in the dataset including: `quantiles`, `min`, `max` and `std`.

`std` helps us understand how spread out the values of that variable are ---> the bigger the `std` the bigger the spread

In [30]:
data.describe()

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
count,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000
mean,95095.000000,46.325960,2.346203,35.928409,3.812580,2.341167,2.455317,8631.953698,1162.814061,7469.139637,0.759941,4404.086304,64.858695,0.712222,0.274894
std,2923.557422,8.016814,1.298908,7.986416,1.554408,1.010622,1.106225,9088.776650,814.987335,9090.685324,0.219207,3397.129254,23.472570,0.238086,0.275691
min,90032.000000,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000
25%,92563.500000,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,359.000000,1324.500000,0.631000,2155.500000,45.000000,0.582000,0.023000
50%,95095.000000,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.000000,3474.000000,0.736000,3899.000000,67.000000,0.702000,0.176000
75%,97626.500000,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11067.500000,1784.000000,9859.000000,0.859000,4741.000000,81.000000,0.818000,0.503000
max,100158.000000,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,18484.000000,139.000000,3.714000,0.999000


Here we can see things like: 

    1) The longest customer in this dataset has been around for 56 months or about 4 years and a half. (Max)
    2) The average number of relationships a customer has is ~4. (Mean and median agree here)
    3) The average credit limit is $8.6K, but the median credit limit is much lower at $4.5K. (signals some skew in this variable) 

In [31]:
print('The average Total_Relationship_Count is', round(np.mean(data['Total_Relationship_Count']), 2), 'and the median is', np.median(data['Total_Relationship_Count']))

The average Total_Relationship_Count is 3.81 and the median is 4.0


In [32]:
print('The average Credit_Limit is $', round(np.mean(data['Credit_Limit']), 2), 'and the median is $', np.median(data['Credit_Limit']))

The average Credit_Limit is $ 8631.95 and the median is $ 4549.0
